# Canopy LW + SW net Energy Balance test per layer

In [1]:
# Add PATH (adapt!)
push!(LOAD_PATH, "/Volumes/cfranken/code/gitHub/LSM-SPAM/src/Leaf/");
push!(LOAD_PATH, "/Volumes/cfranken/code/gitHub/LSM-SPAM/src/Utils/");
#push!(LOAD_PATH, "/Volumes/cfranken/code/gitHub/LSM-SPAM/src/tools/");

# Use Plots:
using Plots
using BenchmarkTools
using Statistics
using Parameters

In [2]:
using Revise
using CanopyRTMod

In [3]:
@unpack wl,wle,wlf, soil = CanopyRTMod;
# Create an array of standard leaves (needs to be in Module later on:
arrayOfLeaves = Array{leafbio{FT}, 1}(undef, CanopyRTMod.canopy.nlayers)
for i = 1:CanopyRTMod.canopy.nlayers
    arrayOfLeaves[i] = leafbio{FT, length(wl), length(wle), length(wlf)}()
    CanopyRTMod.fluspect!(arrayOfLeaves[i], CanopyRTMod.optis)
end

In [4]:
# 4 Different steps to compute Short-Wave RT
CanopyRTMod.computeCanopyGeomProps!(canopy, angles,canOpt)
CanopyRTMod.computeCanopyMatrices!(arrayOfLeaves,canOpt);
CanopyRTMod.RTM_SW!(canopy, canOpt, canRad,sunRad, CanopyRTMod.soil);
CanopyRTMod.deriveCanopyFluxes!(canopy, canOpt, canRad,sunRad, CanopyRTMod.soil, arrayOfLeaves);

In [5]:
# Compute Long Wave (Last term is LW incoming in W m^-2)
Emin,Eplu,netLW =  CanopyRTMod.computeThermalFluxes(arrayOfLeaves, canOpt, canRad, canopy, soil, [Float32(400.0)]);
# Layer Temperatures are here:
@show canRad.T_sun;
@show canRad.T_shade;

F⁻[:, end] = Float32[67.9579]
F⁺[:, end] = Float32[367.72128]
canRad.T_sun = Float32[280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0]
canRad.T_shade = Float32[280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0]


In [6]:
# Net Energy fluxes
@show canRad.intNetLW_shade;
@show canRad.intNetLW_sunlit;
@show canRad.intNetSW_shade;
@show canRad.intNetSW_sunlit;
@show canRad.RnSoilLW;
@show canRad.RnSoil;
#@show 1e6 * canRad.absPAR_sunCab;
@show 1e6 * canRad.absPAR_shadeCab;

canRad.intNetLW_shade = Float32[-33.553574, -40.79455, -46.836624, -51.83625, -55.922886, -59.202374, -61.759624, -63.660862, -64.95533, -65.67654, -65.84316, -65.45952, -64.515686, -62.9872, -60.834496, -58.00182, -54.41583, -49.983654, -44.590523, -38.096764]
canRad.intNetLW_sunlit = Float32[-33.553574, -40.79455, -46.836624, -51.83625, -55.922886, -59.202374, -61.759624, -63.660862, -64.95533, -65.67654, -65.84316, -65.45952, -64.515686, -62.9872, -60.834496, -58.00182, -54.41583, -49.983654, -44.590523, -38.096764]
canRad.intNetSW_shade = Float32[142.27963, 128.6942, 116.84857, 106.50427, 97.45883, 89.541214, 82.60717, 76.5357, 71.22568, 66.593544, 62.570835, 59.102795, 56.14653, 53.670284, 51.652374, 50.080692, 48.952396, 48.273613, 48.059868, 48.336094]
canRad.intNetSW_sunlit = Float32[267.1786, 253.59317, 241.74748, 231.4032, 222.35779, 214.44016, 207.5062, 201.43463, 196.12463, 191.49252, 187.46973, 184.00177, 181.0455, 178.56921, 176.55127, 174.97963, 173.85136, 173.1726, 172.

In [ ]:
plot(Emin', label="Downwelling Thermal")
plot!(Eplu', label="Upwelling Thermal")

In [ ]:
plot(wl, canRad.netSW_sunlit)

In [ ]:
iLAI = 4 /20.
plot(wl,1 ./canOpt.Ps[10] / iLAI * canRad.netSW_sunlit[:,10])
plot!(wl, sunRad.E_direct)

In [ ]:
1 ./canOpt.Ps[10]